<a href="https://colab.research.google.com/github/diegosfc91/EJERCICIOS-DE-CORE-Machine-Learning/blob/main/An%C3%A1lisis_y_Predicci%C3%B3n_de_Precios_de_Autos_(Core).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelos Basados en Arbol

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd
import numpy as np

# Cargar datos desde un archivo CSV
df = pd.read_csv('/content/Automobile_data.csv')

# Reemplazar valores faltantes representados como '?' por NaN
df.replace('?', np.nan, inplace=True)


# Separar las columnas numéricas y categóricas
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = df.select_dtypes(include=['object']).columns

## Para columnas numéricas: Reemplazar valores faltantes con la media
imputer_num = SimpleImputer(strategy='mean')
df[numeric_cols] = imputer_num.fit_transform(df[numeric_cols])

## Para columnas categóricas: Reemplazar valores faltantes con el valor más frecuente
imputer_cat = SimpleImputer(strategy='most_frequent')
df[categorical_cols] = imputer_cat.fit_transform(df[categorical_cols])


## Suponiendo que 'price' es la variable objetivo
df['price'] = pd.to_numeric(df['price'], errors='coerce')  # Asegurar que 'price' sea numérica
X = df.drop(columns=['price'])
y = df['price']

## Crear el codificador y transformar las columnas categóricas
encoder = OneHotEncoder(drop='first', sparse_output=False)  # Evitar multicolinealidad
X_categorical_encoded = pd.DataFrame(
    encoder.fit_transform(df[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols)
)

## Mantener columnas numéricas como están
X_numeric = X[numeric_cols].reset_index(drop=True)

## Combinar numéricas y categóricas transformadas
X_transformed = pd.concat([X_numeric, X_categorical_encoded], axis=1)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=0.2, random_state=43)

# Escalar las características numéricas usando StandardScaler
scaler = StandardScaler()

## Ajustar y transformar el conjunto de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

## Transformar el conjunto de prueba
X_test_scaled = scaler.transform(X_test)


# Regresión Lineal:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Paso 1: Entrenar el modelo de Regresión Lineal
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# Paso 2: Realizar predicciones en el conjunto de prueba
y_pred = model.predict(X_test_scaled)

# Paso 3: Evaluar el rendimiento del modelo
mse = mean_squared_error(y_test, y_pred)  # Error cuadrático medio
r2 = r2_score(y_test, y_pred)            # Coeficiente de determinación

# Paso 4: Mostrar los resultados
print("Evaluación del Modelo de Regresión Lineal:")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")

root_mse = np.sqrt(mse)
print(f"Root Mean Squared Error (RMSE): {root_mse:.2f}")
print("")

# Calcular el promedio de la columna 'price'
average_price = df['price'].mean()

# Mostrar el promedio
print(f"El promedio de los valores en la columna 'price' es: {average_price:.2f}")
print("")

max_price = df['price'].max()
min_price = df['price'].min()

# Mostrar los resultados
print(f"El mayor valor en la columna 'price' es: {max_price:.2f}")
print(f"El menor valor en la columna 'price' es: {min_price:.2f}")

Evaluación del Modelo de Regresión Lineal:
Mean Squared Error (MSE): 6752945.75
Coeficiente de Determinación (R²): 0.80
Root Mean Squared Error (RMSE): 2598.64

El promedio de los valores en la columna 'price' es: 13212.82

El mayor valor en la columna 'price' es: 45400.00
El menor valor en la columna 'price' es: 5118.00


# 4. K-Nearest Neighbors (KNN)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Paso 1: Crear y entrenar el modelo KNN
k = 5  # Número de vecinos
knn_model = KNeighborsRegressor(n_neighbors=k)
knn_model.fit(X_train_scaled, y_train)

# Paso 2: Realizar predicciones en el conjunto de prueba
y_pred_knn = knn_model.predict(X_test_scaled)

# Paso 3: Evaluar el rendimiento del modelo
mse_knn = mean_squared_error(y_test, y_pred_knn)  # Error cuadrático medio
r2_knn = r2_score(y_test, y_pred_knn)            # Coeficiente de determinación

# Paso 4: Mostrar los resultados
print(f"Evaluación del Modelo K-Nearest Neighbors (KNN) con k={k}:")
print(f"Mean Squared Error (MSE): {mse_knn:.2f}")
print(f"Coeficiente de Determinación (R²): {r2_knn:.2f}")

root_mse = np.sqrt(mse_knn)
print(f"Root Mean Squared Error (RMSE): {root_mse:.2f}")
print("")


# Calcular el promedio de la columna 'price'
average_price = df['price'].mean()

# Mostrar el promedio
print(f"El promedio de los valores en la columna 'price' es: {average_price:.2f}")
print("")

max_price = df['price'].max()
min_price = df['price'].min()

# Mostrar los resultados
print(f"El mayor valor en la columna 'price' es: {max_price:.2f}")
print(f"El menor valor en la columna 'price' es: {min_price:.2f}")


Evaluación del Modelo K-Nearest Neighbors (KNN) con k=5:
Mean Squared Error (MSE): 17826944.04
Coeficiente de Determinación (R²): 0.46
Root Mean Squared Error (RMSE): 4222.20

El promedio de los valores en la columna 'price' es: 13212.82

El mayor valor en la columna 'price' es: 45400.00
El menor valor en la columna 'price' es: 5118.00


# Arbol de decision

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Paso 1: Crear y entrenar el modelo de Árbol de Decisión
decision_tree_model = DecisionTreeRegressor(random_state=43)  # Puedes ajustar el random_state para reproducibilidad
decision_tree_model.fit(X_train_scaled, y_train)

# Paso 2: Realizar predicciones en el conjunto de prueba
y_pred_tree = decision_tree_model.predict(X_test_scaled)

# Paso 3: Evaluar el rendimiento del modelo
mse_tree = mean_squared_error(y_test, y_pred_tree)  # Error cuadrático medio
r2_tree = r2_score(y_test, y_pred_tree)             # Coeficiente de determinación

# Paso 4: Mostrar los resultados
print("Evaluación del Modelo de Árbol de Decisión:")
print(f"Mean Squared Error (MSE): {mse_tree:.2f}")
print(f"Coeficiente de Determinación (R²): {r2_tree:.2f}")


root_mse = np.sqrt(mse_tree)
print(f"Root Mean Squared Error (RMSE): {root_mse:.2f}")
print("")


# Calcular el promedio de la columna 'price'
average_price = df['price'].mean()

# Mostrar el promedio
print(f"El promedio de los valores en la columna 'price' es: {average_price:.2f}")
print("")

max_price = df['price'].max()
min_price = df['price'].min()

# Mostrar los resultados
print(f"El mayor valor en la columna 'price' es: {max_price:.2f}")
print(f"El menor valor en la columna 'price' es: {min_price:.2f}")


Evaluación del Modelo de Árbol de Decisión:
Mean Squared Error (MSE): 23165453.76
Coeficiente de Determinación (R²): 0.30
Root Mean Squared Error (RMSE): 4813.05

El promedio de los valores en la columna 'price' es: 13212.82

El mayor valor en la columna 'price' es: 45400.00
El menor valor en la columna 'price' es: 5118.00


# Explicacion de modelos]

Interpretación de la Comparación:
MSE (Mean Squared Error):
El modelo con el MSE más bajo tiene las predicciones más precisas.
R² (Coeficiente de Determinación):
El modelo con el R² más cercano a 1 explica mejor la varianza de los datos.

El mas bajo es la regresion lineal